In [ ]:
import os 
import numpy as np 
import glob 
import cv2
import shutil
import time 
import pickle 
import random 
import json 
from google.colab.patches import cv2_imshow

In [ ]:
boxing_pth = 'Land Application Data/train/'
test_pth = os.path.join(boxing_pth, 'test.txt')


with open(test_pth, 'r') as f: 
  locs = []
  for im in f.readlines():
    locs.append('loc_' + im.split(os.sep)[-1].split('_')[1])

locs = np.unique(locs)

# PR & ROC Curves

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt 

In [ ]:
# YOLOv5 Classification results 
boxing_pth = 'Land Application Data/train/'

results = '/DATAPATH/runs/detect/exp/'
label_dir = os.path.join(results, 'labels')

# Get positives 
with open(os.path.join(boxing_pth, 'test.txt'), 'r') as f: 
  positives = [l.strip().split(os.sep)[-1] for l in f.readlines()]

y_true, y_probas = [], []
for im in glob.glob(os.path.join(results, '*png')): 
  im_name = im.split(os.sep)[-1]
  if im_name in positives: 
    y_true.append(1)
  else: 
    y_true.append(0)

  try: 
    with open(os.path.join(label_dir, im_name.replace('.png', '.txt')), 'r') as f: 
      detections = [l.strip() for l in f.readlines()]
      confs = [float(d.split(' ')[-1]) for d in detections]
      y_probas.append(np.max(confs))
  except FileNotFoundError: 
    y_probas.append(0) 
  

In [ ]:
# Yolov5 and RCNN instance detection results from inference script
with open('./instance_pr.p', 'rb') as f: 
  d = pickle.load(f)
  recall_event = d['recalls']
  inds = np.argsort(recall_event)
  recall_event = np.array(recall_event)[inds]
  precision_event = np.array(d['precisions'])[inds]



In [ ]:
precision_cls, recall_cls, thresh = precision_recall_curve(y_true, y_probas)

In [ ]:
def probas2pred(probas, t): 
  return [1 if p >= t else 0 for p in probas]

f1_yolo = fbeta_score(y_true, probas2pred(y_probas, 0.5), beta=1)
print('F1 scores', f1_cnn, f1_dual, f1_rcnn, f1_yolo)

f2_yolo = fbeta_score(y_true, probas2pred(y_probas, 0.5), beta=2)
print('F2 scores', f2_cnn, f2_dual, f2_rcnn, f2_yolo)

In [ ]:
precision_cls, recall_cls, thresh = precision_recall_curve(y_true, y_probas)
precision_cls[precision_cls < 0.5] = 0.5
plt.plot(recall_cls, precision_cls, c='tab:olive', lw=3, label=f'YOLOv5 (AUC {auc(recall_cls, precision_cls):0.2f})')
plt.hlines(y=0.5, xmin=0, xmax=1, color='k', ls='--', label='no skill')
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.title(f'Does the image contain application?')
plt.legend(loc='lower left', bbox_to_anchor=(0.05, 0.2))

In [ ]:
plt.plot(recall_cls, precision_cls, c='tab:olive', lw=3, label=f'YOLO (AUC {auc(recall_cls, precision_cls):0.2f})')
plt.hlines(y=np.mean(y_true), xmin=0, xmax=1, color='k', ls='--', label='no skill')
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.title(f'Task 1: Classification')
plt.legend()
plt.show()

plt.plot(recall_event, precision_event, c='green', lw=3, label=f'YOLO AUC {auc(recall_event, precision_event):0.2f}')
plt.legend()
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.title(f'Task 2: Event Detection')
plt.show()


fpr_cls, tpr_cls, thresh = roc_curve(y_true, y_probas)
plt.plot(fpr_cls, tpr_cls, lw=3, c='tab:olive', label=f'YOLO classificaton (AUC {auc(fpr_cls, tpr_cls):0.2f})')
plt.plot([0,1], [0,1], ls='--', c='k', label='random classifier')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.title('Task 1: Classification')
plt.show()
